In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

### 回傳Leaderboards上某頁數的全部召喚師ID(伺服器region =1:KR, =2:NA, =3:EUW, =4:EUNE)

In [ ]:
def get_ID(region:int, page:int)->list:
    container = []
    dict = {1:"www", 2:"na", 3:"euw", 4:"eune"}
    soup = BeautifulSoup(requests.get('https://' + dict[region] + '.op.gg/ranking/ladder/page=' + str(page)).text, 'html.parser')      
    if page == 1:
        for i in soup.find_all("a", class_="ranking-highest__name"):
            container += re.findall('<a.*?>(.*?)</a>', str(i), re.S)
    for i in soup.find_all("td", class_="select_summoner ranking-table__cell ranking-table__cell--summoner"):
        container += re.findall('<span>(.*?)</span>', str(i), re.S)
    return container  

### 回傳某召喚師ID的最近二十場內有效牌位對戰英雄(左上,左野,左中,左下,左輔,右上,右野,右中,右下,右輔,1=左贏0=左輸)

In [ ]:
def get_match(summoner_id:str)->pd.DataFrame:
    soup = BeautifulSoup(requests.get('https://www.op.gg/summoner/userName=' + summoner_id.replace(" ", "+")).text, 'html.parser')
    container = []
    for i in soup.find_all("div", class_="Image16"):
        container += re.findall('<div.*?>(.*?)</div>', str(i), re.S)
    df = pd.DataFrame(np.reshape(container, (-1, 10)))
    df[10] = ""
    for i, j in enumerate(soup.select('div.GameSettingInfo > div.ChampionName')):
        df.iloc[i, 10] = re.findall('<a.*?>(.*?)</a>', str(j), re.S)[0]
    df[11] = ""
    for i, j in enumerate(soup.find_all("div", class_="GameType")):
        df.iloc[i, 11] = re.findall('<div.*?>(.*?)</div>', str(j), re.S)[0]
    df[12] = ""
    for i, j in enumerate(soup.find_all("div", class_="GameResult")):
        df.iloc[i, 12] = re.findall('<div.*?>(.*?)</div>', str(j), re.S)[0]
    df = df[~df[12].str.contains("Remake") & df[11].str.contains("Ranked")]
    df['result'] = np.where((((df[0]==df[10])|(df[1]==df[10])|(df[2]==df[10])|(df[3]==df[10])|(df[4]==df[10]))&(df[12].str.contains("Victory")))|
                            (((df[5]==df[10])|(df[6]==df[10])|(df[7]==df[10])|(df[8]==df[10])|(df[9]==df[10]))&(df[12].str.contains("Defeat"))), 1, 0)
    df.drop(df.columns[[10, 11, 12]], axis = 1, inplace = True)
    return df

In [ ]:
# 回傳
list = []
dict = {}
r = requests.get('https://www.op.gg/champion/statistics')
soup = BeautifulSoup(r.text, 'html.parser')
for link in soup.select('a[href^="/champion/"]'):
    list.append(link.get('href'))
for i in list[1:148]:
    r = requests.get('https://www.op.gg' + str(i))
    a = re.findall('<h1.*?>(.*?)</h1>', str(BeautifulSoup(r.text, 'html.parser').find_all('h1', class_="champion-stats-header-info__name")), re.S)[0]
    b = str(BeautifulSoup(r.text, 'html.parser').find_all('script')).partition('championId: ')[2].partition('\n')[0]
    dict.setdefault(a,b)
print(dict)

### 回傳第一頁到第n頁的所有召喚師對位勝率和結果，並且過濾掉可能重複的對戰

In [ ]:
def get_many_match(n:int)->pd.DataFrame:
    #df_output = pd.DataFrame() 
    #for i in range(1, n + 1):
      #  for j in get_ID(1, i):
     #       df_output = df_output.append(get_match(j))
    #df_output.drop_duplicates(keep = 'first', inplace = True)
    df = get_match("znhy")
    list = []
    container = []
    r = requests.get('https://www.op.gg/champion/statistics')
    soup = BeautifulSoup(r.text, 'html.parser')
    for i in soup.find_all("div", class_="champion-index__champion-item__name"):
        list += re.findall('<div.*?>(.*?)</div>', str(i), re.S)
    for summoner in list:
        for key, value in {0:"top", 1:"jungle", 2:"mid", 3:"bot", 4:"support"}.items():
            url = 'https://www.op.gg/champion/' + str(summoner) + '/statistics/' + value + '/matchup'
            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'html.parser')
            for i, j in enumerate(soup.select('div.champion-matchup-list__champion > span')):
                if i % 2 == 0:
                    container += str(key)
                    container.append(summoner)
                container.append(re.findall('<span.*?>(.*?)</span>', str(j), re.S)[0].replace("\n", "").replace("\t", "").replace("%", ""))
    df = df.reset_index(drop=True)
    df["top"] = ""
    df["jungle"] = ""
    df["mid"] = ""
    df["bot"] = ""
    df["sup"] = ""
    for index, row in df.iterrows():
        for i in np.reshape(container, (-1, 4)).tolist():
            if i[0] == "0" and i[1] == row[0] and i[2] == row[5]:
                df.loc[index, "top"] = float(i[3]) * 0.01
            if i[0] == "1" and i[1] == row[1] and i[2] == row[6]:
                df.loc[index, "jungle"] = float(i[3]) * 0.01
            if i[0] == "2" and i[1] == row[2] and i[2] == row[7]:
                df.loc[index, "mid"] = float(i[3]) * 0.01
            if i[0] == "3" and i[1] == row[3] and i[2] == row[8]:
                df.loc[index, "bot"] = float(i[3]) * 0.01
            if i[0] == "4" and i[1] == row[4] and i[2] == row[9]:
                df.loc[index, "sup"] = float(i[3]) * 0.01
    df.replace('', 0.5, inplace=True)
    return df
get_many_match(1)